In [1]:
import sqlite3 as sql
import os
import csv
from sqlite3 import Error

In [8]:
os.getcwd()
os.chdir("../data/raw/outputs/")
input_folder = os.fspath(os.getcwd())
os.chdir("../../processed/")
output_folder = os.fspath(os.getcwd())

FileNotFoundError: [Errno 2] No such file or directory: '../data/raw/outputs/'

In [33]:
input_folder

'/Users/alanzhang/Documents/DSC 180B/Intel-capstone/data/raw/outputs'

In [41]:
".db" in "ucsd_intel_capstone_180A_grp_2-000036.db"

True

In [48]:
output_folder = '/Users/alanzhang/Documents/DSC 180B/Intel-capstone/data/raw/'

In [49]:
for db in os.listdir(input_folder):
    if ".db" not in db:
        continue
    number = db.split("-")[1].strip("0.db")
    new_name = f"df{number}.csv"
    try:
        input_file_path = os.path.join(input_folder, db)
        conn=sql.connect(input_file_path)
        cursor = conn.cursor()
        cursor.execute("SELECT * FROM COUNTERS_STRING_TIME_DATA")
        print("Exporting data into CSV............")
        dirpath = os.path.join(output_folder, new_name)
        with open(dirpath, "w") as csv_file:
            csv_writer = csv.writer(csv_file, delimiter="\t")
            csv_writer.writerow([i[0] for i in cursor.description])
            csv_writer.writerows(cursor)
        print("Data exported Successfully into {}".format(dirpath))

    except Error as e:
        print(e)

    # Close database connection
    finally:
        conn.close()

Exporting data into CSV............
Data exported Successfully into /Users/alanzhang/Documents/DSC 180B/Intel-capstone/data/raw/df36.csv
Exporting data into CSV............
Data exported Successfully into /Users/alanzhang/Documents/DSC 180B/Intel-capstone/data/raw/df71.csv
Exporting data into CSV............
Data exported Successfully into /Users/alanzhang/Documents/DSC 180B/Intel-capstone/data/raw/df53.csv
Exporting data into CSV............
Data exported Successfully into /Users/alanzhang/Documents/DSC 180B/Intel-capstone/data/raw/df12.csv
Exporting data into CSV............
Data exported Successfully into /Users/alanzhang/Documents/DSC 180B/Intel-capstone/data/raw/df43.csv
Exporting data into CSV............
Data exported Successfully into /Users/alanzhang/Documents/DSC 180B/Intel-capstone/data/raw/df61.csv
Exporting data into CSV............
Data exported Successfully into /Users/alanzhang/Documents/DSC 180B/Intel-capstone/data/raw/df26.csv
Exporting data into CSV............
Data 

In [25]:
os.getcwd()

'/Users/alanzhang/Documents/DSC 180B/Intel-capstone/data/processed'

In [27]:
os.getcwd()

'/Users/alanzhang/Documents/DSC 180B/Intel-capstone/data/processed'

In [28]:
input_folder

'/Users/alanzhang/Documents/DSC 180B/Intel-capstone/data/raw/outputs'

In [ ]:
datasets = []
# _, output_folder = get_input_output_folder()
for csv in os.listdir(output_folder):
    if ".csv" not in csv:
        continue
    curr_file_path = os.path.join(output_folder, csv)
    df = pd.read_csv(curr_file_path)
    temp = df[df['ID_INPUT'] == 3][['MEASUREMENT_TIME', 'VALUE']].reset_index(drop=True)
    temp = temp.rename(columns={'MEASUREMENT_TIME': 'Start', 'VALUE': 'Value'})
    temp['Start'] = pd.to_datetime(temp['Start']).dt.tz_localize(tz='GMT+0').dt.tz_convert('America/Los_Angeles').dt.tz_localize(None)
    temp['End'] = temp['Start'].shift(-1)
    temp['Duration'] = (temp['Start'].shift(-1) - temp['Start'])
    temp = temp.drop(len(temp)-1)
    datasets.append(temp)
output_dataset = pd.concat(datasets).reset_index(drop=True)
# print(output_dataset['Duration'].dtype)
output_dataset.to_csv(file_path, index=False)·